# VAE Latent Dimension Evaluation on GPU

In [ ]:
!pip install xlrd

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

# Load the original data
data = pd.read_excel('/kaggle/input/dataaa/default of credit card clients.xls')

# Drop the first row and reset the index if needed
data = data.drop(index=0)
data.reset_index(drop=True, inplace=True)

# Convert columns to numeric
data = data.apply(pd.to_numeric, errors='coerce').fillna(0)

In [ ]:
def calculate_periodic_weighted_loss(original, generated):
    sine_weights = torch.sin(original * np.pi)
    weighted_diff = torch.sum(sine_weights * (original - generated).pow(2))
    return weighted_diff.item()

In [ ]:
def evaluate_latent_dimensions(data, latent_dims, input_dim, epochs=50, batch_size=64, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    results = {}

    for latent_dim in latent_dims:
        class Encoder(nn.Module):
            def __init__(self, input_dim, latent_dim):
                super(Encoder, self).__init__()
                self.fc1 = nn.Linear(input_dim, 256)
                self.fc2 = nn.Linear(256, 128)
                self.fc3 = nn.Linear(128, 64)
                self.batch_norm1 = nn.BatchNorm1d(256)
                self.batch_norm2 = nn.BatchNorm1d(128)
                self.dropout = nn.Dropout(0.3)
                self.z_mean = nn.Linear(64, latent_dim)
                self.z_log_var = nn.Linear(64, latent_dim)

            def forward(self, x):
                x = F.relu(self.batch_norm1(self.fc1(x)))
                x = F.relu(self.batch_norm2(self.fc2(x)))
                x = self.dropout(F.relu(self.fc3(x)))
                z_mean = self.z_mean(x)
                z_log_var = self.z_log_var(x)
                return z_mean, z_log_var

        class Decoder(nn.Module):
            def __init__(self, latent_dim, input_dim):
                super(Decoder, self).__init__()
                self.fc1 = nn.Linear(latent_dim, 64)
                self.fc2 = nn.Linear(64, 128)
                self.fc3 = nn.Linear(128, 256)
                self.batch_norm1 = nn.BatchNorm1d(64)
                self.batch_norm2 = nn.BatchNorm1d(128)
                self.dropout = nn.Dropout(0.3)
                self.fc4 = nn.Linear(256, input_dim)

            def forward(self, z):
                x = F.relu(self.batch_norm1(self.fc1(z)))
                x = F.relu(self.batch_norm2(self.fc2(x)))
                x = self.dropout(F.relu(self.fc3(x)))
                x = torch.sigmoid(self.fc4(x))
                return x

        class VAE(nn.Module):
            def __init__(self, input_dim, latent_dim):
                super(VAE, self).__init__()
                self.encoder = Encoder(input_dim, latent_dim)
                self.decoder = Decoder(latent_dim, input_dim)

            def forward(self, x):
                z_mean, z_log_var = self.encoder(x)
                z = reparameterize(z_mean, z_log_var)
                reconstruction = self.decoder(z)
                return reconstruction, z_mean, z_log_var

        def reparameterize(z_mean, z_log_var):
            std = torch.exp(0.5 * z_log_var)
            epsilon = torch.randn_like(std)
            return z_mean + std * epsilon

        def periodic_weighted_loss(original, reconstruction, z_mean, z_log_var, input_dim):
            reconstruction_loss = F.mse_loss(reconstruction, original, reduction='sum')
            kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - torch.exp(z_log_var))
            sine_weights = torch.sin(original * np.pi)
            weighted_diff = torch.sum(sine_weights * (original - reconstruction).pow(2))
            total_loss = (reconstruction_loss + kl_loss + weighted_diff) / input_dim
            return total_loss

        normalized_data = (data - data.mean()) / data.std()
        normalized_data = torch.tensor(normalized_data.values, dtype=torch.float32).to(device)
        dataset = TensorDataset(normalized_data)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        vae = VAE(input_dim, latent_dim).to(device)
        optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)
        vae.train()
        total_loss = 0
        for epoch in range(epochs):
            for batch in dataloader:
                x = batch[0].to(device)
                optimizer.zero_grad()
                reconstruction, z_mean, z_log_var = vae(x)
                loss = periodic_weighted_loss(x, reconstruction, z_mean, z_log_var, input_dim)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
        results[latent_dim] = total_loss / len(dataset)

    return results

In [ ]:
latent_dims = [2, 4, 6, 8, 10, 12, 15]
input_dim = data.shape[1]
epochs = 50
results = evaluate_latent_dimensions(data, latent_dims, input_dim, epochs=epochs)

for latent_dim, loss in results.items():
    print(f"Latent Dimension: {latent_dim}, Loss: {loss:.4f}")